In [ ]:
# Day 3
from typing import List
import pathlib

example = [
    "00100",
    "11110",
    "10110",
    "10111",
    "10101",
    "01111",
    "00111",
    "11100",
    "10000",
    "11001",
    "00010",
    "01010",
]
example_txt = """00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010"""


def get_bits_by_index(index: int, xs: List[str]) -> List[bool]:
    return [bool(int(x[index])) for x in xs]


assert get_bits_by_index(0, example) == [0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0]
assert get_bits_by_index(1, example) == [0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1]


def get_most_common_bit(index: int, xs: List[str]) -> bool:
    bits = get_bits_by_index(index, xs)
    bit_set_ratio = sum(bits) / len(bits)
    return bit_set_ratio >= 0.5


assert get_most_common_bit(0, example) == True
assert get_most_common_bit(1, example) == False


def get_most_common_bits(xs: List[str]) -> List[bool]:
    return [get_most_common_bit(y, xs) for y in range(len(xs[0]))]


assert get_most_common_bits(example) == [1, 0, 1, 1, 0]


def binarray2dec(xs: List[bool]) -> int:
    return sum(int(x) * 2 ** i for i, x in enumerate(reversed(xs)))


assert binarray2dec([True]) == 1
assert binarray2dec([True, False]) == 2
assert binarray2dec([True, True]) == 3


def invert_binarray(xs: List[bool]) -> List[bool]:
    return [not x for x in xs]


assert invert_binarray([True]) == [False]
assert invert_binarray([False, True]) == [True, False]


def gamma_rate(xs: List[str]) -> int:
    return binarray2dec(get_most_common_bits(xs))


assert gamma_rate(example) == 22


def epsilon_rate(xs: List[str]) -> int:
    return binarray2dec(invert_binarray(get_most_common_bits(xs)))


assert epsilon_rate(example) == 9


def solve_day3_part1(xs: List[str]) -> int:
    return gamma_rate(xs) * epsilon_rate(xs)


assert solve_day3_part1(example) == 198


def parse_day3(s: str) -> List[str]:
    return s.splitlines()


assert parse_day3(example_txt) == example

p = pathlib.Path("data/AoC_2021_day_03.txt")
assert solve_day3_part1(parse_day3(p.read_text())) == 1131506


def find_oxygen_generator_rating(xs: List[str]) -> int:
    filtered = xs.copy()
    idx = 0
    while len(filtered) > 1 and idx < len(xs):
        most_common = get_most_common_bit(idx, filtered)
        filtered = [y for y in filtered if int(y[idx]) == most_common]
        idx += 1
    return binarray2dec(list(map(int, filtered[0])))


assert find_oxygen_generator_rating(example) == 23


def find_co2_scrubber_rating(xs: List[str]) -> int:
    filtered = xs.copy()
    idx = 0
    while len(filtered) > 1 and idx < len(xs):
        least_common = not get_most_common_bit(idx, filtered)
        filtered = [y for y in filtered if int(y[idx]) == least_common]
        idx += 1
    return binarray2dec(list(map(int, filtered[0])))


assert find_co2_scrubber_rating(example) == 10


def solve_day3_part2(xs: List[str]) -> int:
    return find_oxygen_generator_rating(xs) * find_co2_scrubber_rating(xs)


assert solve_day3_part2(example) == 230


assert solve_day3_part2(parse_day3(p.read_text())) == 7863147


In [ ]:
# Day 4
from typing import List, Tuple, TypeVar
import pathlib
from copy import deepcopy

d4_example_txt = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7"""

Board = TypeVar("Board")  # List[List[Tuple[int, bool]]]


def parse_day4(s: str) -> Tuple[List[int], List[Board]]:
    chunks = s.replace("  ", " ").split("\n\n")
    draws = list(map(int, chunks[0].split(",")))
    boards = []
    for chunk in chunks[1:]:
        board = []
        for line in chunk.splitlines():
            board.append(list(map(lambda x: (int(x), False), line.split())))
        boards.append(board)
    return (draws, boards)


assert parse_day4(d4_example_txt)[0][:4] == [7, 4, 9, 5]
assert parse_day4(d4_example_txt)[1][0][0] == [
    (22, False),
    (13, False),
    (17, False),
    (11, False),
    (0, False),
]
d4_example = parse_day4(d4_example_txt)


def check_row(board: Board, row: int) -> bool:
    for num in board[row]:
        if not num[1]:
            return False
    return True


assert check_row(d4_example[1][0], 0) == False
d4_example_row_match = [[(x, True) for x in d4_example[1][0][0]]] + d4_example[1][0][1:]
assert check_row(d4_example_row_match, 0) == True


def check_column(board: Board, column: int) -> bool:
    col = [x[column] for x in board]
    for num in col:
        if not num[1]:
            return False
    return True


assert check_column(d4_example[1][0], 0) == False
d4_example_col_match = deepcopy(d4_example[1][0])
for row in d4_example_col_match:
    row[0] = (row[0][0], True)
assert check_column(d4_example_col_match, 0) == True


def draw(n: int, boards: List[Board]) -> List[Board]:
    for board in boards:
        for row in board:
            for i, num in enumerate(row):
                if num[0] == n:
                    row[i] = (n, True)
    return boards


assert draw(22, [deepcopy(d4_example[1][0])])[0][0] == [
    (22, True),
    (13, False),
    (17, False),
    (11, False),
    (0, False),
]


def find_winner(draws, boards):
    for n in draws:
        draw(n, boards)
        for board in boards:
            for i in range(5):
                if check_row(board, i) or check_column(board, i):
                    return (n, board)
    assert False, "No winner?"


assert find_winner(*d4_example) == (24, d4_example[1][2])


def sum_unmarked(board: Board) -> int:
    total = 0
    for row in board:
        for num in row:
            if not num[1]:
                total += num[0]
    return total


assert sum_unmarked(find_winner(*d4_example)[1]) == 188


def solve_day4_part1(draws_and_boards):
    n, b = find_winner(*draws_and_boards)
    return n * sum_unmarked(b)


assert solve_day4_part1(parse_day4(d4_example_txt)) == 4512

p = pathlib.Path("data/AoC_2021_day_04.txt")
assert solve_day4_part1(parse_day4(p.read_text())) == 58374


def find_last_winner(draws, boards):
    winners = [False for b in boards]
    for n in draws:
        draw(n, boards)
        for b, board in enumerate(boards):
            for i in range(5):
                if check_row(board, i) or check_column(board, i):
                    winners[b] = True
                    if all(winners):
                        return (n, board)
    assert False, "No winner?"


def solve_day4_part2(draws_and_boards):
    n, b = find_last_winner(*draws_and_boards)
    return n * sum_unmarked(b)


assert solve_day4_part2(parse_day4(d4_example_txt)) == 1924

p = pathlib.Path("data/AoC_2021_day_04.txt")
assert solve_day4_part2(parse_day4(p.read_text())) == 11377


In [ ]:
# Day 5
from collections import defaultdict

example_txt = """0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2"""


def parse_day5(s: str) -> List[List[Tuple[int, int]]]:
    return [
        [tuple(map(int, pos.split(","))) for pos in line.split(" -> ")]
        for line in s.splitlines()
    ]


assert parse_day5(example_txt)[0] == [(0, 9), (5, 9)]


def filter_straight_lines(
    lines: List[List[Tuple[int, int]]]
) -> List[List[Tuple[int, int]]]:
    return list(
        filter(lambda line: line[0][0] == line[1][0] or line[0][1] == line[1][1], lines)
    )


assert len(filter_straight_lines(parse_day5(example_txt))) == 6
assert filter_straight_lines(parse_day5(example_txt))[0] == [(0, 9), (5, 9)]


def expand(line):
    x1 = line[0][0]
    y1 = line[0][1]
    x2 = line[1][0]
    y2 = line[1][1]
    if x1 == x2:
        return [(x1, y) for y in range(min(y2, y1), max(y2, y1) + 1)]
    elif y1 == y2:
        return [(x, y1) for x in range(min(x2, x1), max(x2, x1) + 1)]
    else:
        # Lines can only be +/- 45 degrees
        range_x = range(min(x1, x2), max(x1, x2) + 1)
        range_y = range(min(y1, y2), max(y1, y2) + 1)
        if x2 < x1:
            range_x = reversed(range_x)
        if y2 < y1:
            range_y = reversed(range_y)
        return list(zip(range_x, range_y))


assert expand([(1, 2), (1, 4)]) == [(1, 2), (1, 3), (1, 4)]
assert expand([(1, 1), (3, 3)]) == [(1, 1), (2, 2), (3, 3)]
assert expand([(9, 7), (7, 9)]) == [(9, 7), (8, 8), (7, 9)]


def count_overlaps(lines: List[List[Tuple[int, int]]]) -> int:
    floor_map = defaultdict(int)
    for line in lines:
        for point in expand(line):
            floor_map[point] += 1
    return len(list(filter(lambda x: x > 1, floor_map.values())))


assert count_overlaps(filter_straight_lines(parse_day5(example_txt))) == 5

p = pathlib.Path("data/AoC_2021_day_05.txt")

# Part 1
assert count_overlaps(filter_straight_lines(parse_day5(p.read_text()))) == 5124

# Part 2
assert count_overlaps(parse_day5(p.read_text())) == 19771


In [ ]:
# Day 6

example_state = [3, 4, 3, 1, 2]


def simulate(initial_state, days=80):
    shoal = [0 for _ in range(7)]
    for f in initial_state:
        shoal[f] += 1
    incubator = [0 for _ in range(2)]
    for _ in range(days):
        fish_count = shoal.pop(0)
        larvae_count = incubator.pop(0)
        shoal.append(fish_count + larvae_count)
        incubator.append(fish_count)
    return sum(shoal + incubator)


assert simulate(example_state) == 5_934

p = pathlib.Path("data/AoC_2021_day_06.txt")
assert simulate(map(int, p.read_text().split(","))) == 372_300

assert simulate(example_state, days=256) == 26_984_457_539

assert simulate(map(int, p.read_text().split(",")), days=256) == 1675781200288


In [ ]:
# Day 7

d7_example = [16, 1, 2, 0, 4, 2, 7, 1, 2, 14]
p = pathlib.Path("data/AoC_2021_day_07.txt")


def solve_day7_part1(positions):
    return sorted(
        (
            (i, sum([abs(x - i) for x in positions]))
            for i in range(min(positions), max(positions))
        ),
        key=lambda x: x[1],
    )[0][1]


assert solve_day7_part1(d7_example) == 37
assert solve_day7_part1(list(map(int, p.read_text().split(",")))) == 342730


def spend_fuel(x, y):
    return sum(range(1, max(x, y) - min(x, y) + 1))


assert spend_fuel(16, 5) == 66
assert spend_fuel(1, 5) == 10


def spend_fuel2(x, y):
    n = max(x, y) - min(x, y)
    return n * (n + 1) / 2


assert spend_fuel2(16, 5) == 66
assert spend_fuel2(1, 5) == 10


def solve_day7_part2(positions):
    return sorted(
        (
            (i, sum(spend_fuel2(x, i) for x in positions))
            for i in range(min(positions), max(positions))
        ),
        key=lambda x: x[1],
    )[0][1]


assert solve_day7_part2(d7_example) == 168

assert solve_day7_part2(list(map(int, p.read_text().split(",")))) == 92335207


In [ ]:
# Day 8
import pathlib
from typing import List, Tuple, Dict, Set, Optional
from collections import Counter
import itertools as it

d8_example_txt = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce"""

short_example = "acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf"

# length: digit
unique_digits = {3: 7, 4: 4, 2: 1, 7: 8}
# count: segment
unique_segments = {4: "e", 6: "b", 9: "f"}
# 0..9 correctly indexed
normal_digit_patterns = [
    "abcefg",
    "cf",
    "acdeg",
    "acdfg",
    "bcdf",
    "abdfg",
    "abdefg",
    "acf",
    "abcdefg",
    "abcdfg",
]
normal_digit_pattern_sets = [set(x) for x in normal_digit_patterns]


def parse_day8(s: str) -> List[Tuple[List[Set[str]]]]:
    return [
        tuple([set(s) for s in part.split()] for part in line.split(" | "))
        for line in s.splitlines()
    ]


parsed_short_example = parse_day8(short_example)
assert parsed_short_example[0][0][0] == set("acedgfb")
assert parsed_short_example[0][1][0] == set("cdfeb")


def solve_day8_part1(s: str):
    total = 0
    for line in parse_day8(s):
        _, output = line
        for digit in output:
            if len(digit) in unique_digits.keys():
                total += 1
    return total


assert solve_day8_part1(d8_example_txt) == 26

p = pathlib.Path("data/AoC_2021_day_08.txt")
assert solve_day8_part1(p.read_text()) == 301


def find_missing_translation(
    pattern: Set[str], translation: Dict[str, str]
) -> Optional[Tuple[str, str]]:
    normal_digit = normal_digit_pattern_sets[unique_digits[len(pattern)]]
    new_letter = (pattern - set(translation.keys())).pop()
    mapped_letter = (normal_digit - set(translation.values())).pop()
    return (new_letter, mapped_letter)


def build_translation(patterns: List[str]) -> Dict[str, str]:
    translation = {}
    for k, v in Counter(it.chain(*patterns)).items():
        if v in unique_segments:
            translation[k] = unique_segments[v]

    unique_patterns = sorted(
        filter(lambda pat: len(pat) in unique_digits.keys(), patterns), key=len
    )
    for pat in unique_patterns:
        letters = find_missing_translation(pat, translation)
        translation[letters[0]] = letters[1]

    return translation


short_example_translation = dict(
    sorted(build_translation(parsed_short_example[0][0]).items())
)
expected_translation = dict(
    sorted(
        {"d": "a", "e": "b", "a": "c", "f": "d", "g": "e", "b": "f", "c": "g"}.items()
    )
)
assert (
    short_example_translation == expected_translation
), f"{short_example_translation} != {expected_translation}"


def get_digit(s: Set[str], translation: Dict[str, str]) -> int:
    if len(s) in unique_digits.keys():
        return unique_digits[len(s)]

    normalized_digit = "".join(sorted("".join(s).translate(str.maketrans(translation))))
    return normal_digit_patterns.index(normalized_digit)


assert [
    get_digit(pat, short_example_translation) for pat in parsed_short_example[0][0]
] == [8, 5, 2, 3, 7, 9, 6, 4, 0, 1]
assert [
    get_digit(part, short_example_translation) for part in parsed_short_example[0][1]
] == [5, 3, 5, 3]


def solve_day8_part2(s: str):
    lines = parse_day8(s)
    total = 0
    for patterns, output in lines:
        translation = build_translation(patterns)
        total += int("".join([str(get_digit(part, translation)) for part in output]))
    return total


assert solve_day8_part2(short_example) == 5353
assert solve_day8_part2(d8_example_txt) == 61229
assert solve_day8_part2(p.read_text()) == 908067


In [ ]:
# Day 9
from typing import List, Set, Tuple
from operator import mul
from functools import reduce
import itertools as it
import pathlib
from collections import defaultdict

d9_example_txt = """2199943210
3987894921
9856789892
8767896789
9899965678"""


def parse_day9(s: str) -> List[List[int]]:
    return [[int(x) for x in line] for line in s.splitlines()]


parsed_d9_example = parse_day9(d9_example_txt)
assert parsed_d9_example[0][0] == 2
assert parsed_d9_example[1][0] == 3


def find_low_points(heightmap: List[List[int]]) -> List[int]:
    low_points = []
    for i, line in enumerate(heightmap):
        previous_line = it.repeat(9, len(line)) if i == 0 else heightmap[i - 1]
        next_line = (
            it.repeat(9, len(line)) if i == (len(heightmap) - 1) else heightmap[i + 1]
        )
        points = zip(previous_line, [9] + line[:-1], line, line[1:] + [9], next_line)
        low_points += [
            ((j, i), x[2])
            for j, x in enumerate(points)
            if all(x[2] < x[k] for k in [0, 1, 3, 4])
        ]
    return list(low_points)


assert len(find_low_points(parsed_d9_example)) == 4
assert [x[1] for x in find_low_points(parsed_d9_example)] == [1, 0, 5, 5]


def solve_day9_part1(s: str) -> int:
    return sum(x[1] + 1 for x in find_low_points(parse_day9(s)))


assert solve_day9_part1(d9_example_txt) == 15

p = pathlib.Path("data/AoC_2021_day_09.txt")
assert solve_day9_part1(p.read_text()) == 444


def find_basin_size(heightmap, x, y, checked=[]):
    if heightmap[y][x] == 9 or (x, y) in checked:
        return 0

    checked.append((x, y))
    basin_size = 1

    new_x = x + 1
    if new_x < len(heightmap[y]):
        basin_size += find_basin_size(heightmap, new_x, y)

    new_x = x - 1
    if new_x >= 0:
        basin_size += find_basin_size(heightmap, new_x, y)

    new_y = y + 1
    if new_y < len(heightmap):
        basin_size += find_basin_size(heightmap, x, new_y)

    new_y = y - 1
    if new_y >= 0:
        basin_size += find_basin_size(heightmap, x, new_y)

    return basin_size


def find_basin_sizes(heightmap):
    low_points = find_low_points(heightmap)
    return sorted([find_basin_size(heightmap, *point[0]) for point in low_points])


def solve_day9_part2(s: str) -> int:
    return reduce(mul, find_basin_sizes(parse_day9(s))[-3:])


assert solve_day9_part2(d9_example_txt) == 1134


assert solve_day9_part2(p.read_text()) == 1168440


In [ ]:
# Day 10

d10_example_text = """[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]"""


def parse_day10(s: str) -> List[str]:
    return s.splitlines()


score_lookup = {
    ")": 3,
    "]": 57,
    "}": 1197,
    ">": 25137,
    "": 0,
}

opening_braces = [
    "(",
    "[",
    "{",
    "<",
]
closing_braces = list(score_lookup.keys())


def check_line(line: str) -> str:
    stack = []
    for c in line:
        if c in opening_braces:
            stack.append(c)
        elif c == closing_braces[opening_braces.index(stack[-1])]:
            stack.pop()
        else:
            return c
    return ""


assert check_line(r"[<>({}){}[([])<>]]") == ""
assert check_line(r"(]") == r"]"


def solve_day10_part1(s: str):
    return sum(score_lookup[c] for c in [check_line(line) for line in parse_day10(s)])


assert solve_day10_part1(d10_example_text) == 26397

p = pathlib.Path("data/AoC_2021_day_10.txt")
assert solve_day10_part1(p.read_text()) == 193275


def complete_line(line: str) -> str:
    stack = []
    inserted = []
    for c in line:
        if c in opening_braces:
            stack.append(c)
        else:
            stack.pop()
    for s in reversed(stack):
        inserted.append(closing_braces[opening_braces.index(s)])
    return inserted


def solve_day10_part2(s: str):
    points = []
    for completion in [
        complete_line(line) for line in parse_day10(s) if not check_line(line)
    ]:
        total = 0
        for c in completion:
            total *= 5
            total += closing_braces.index(c) + 1
        points.append(total)
    points = sorted(points)
    middle = points[int(len(points) / 2)]
    return middle


assert solve_day10_part2(d10_example_text) == 288957

assert solve_day10_part2(p.read_text()) == 2429644557


In [ ]:
# Day 11
from typing import List
from pprint import pprint

d11_example_txt = """5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526"""


def parse_day11(s: str) -> List[List[int]]:
    return [[int(x) for x in line] for line in s.splitlines()]


assert parse_day11(d11_example_txt)[0][0] == 5
assert parse_day11(d11_example_txt)[-1][-1] == 6


def in_range(x, y, matrix):
    return 0 <= y < 10 and 0 <= x < 10


def get_adjacent(x, y):
    return [
        (x, y - 1),  # down
        (x, y + 1),  # up
        (x - 1, y),  # left
        (x - 1, y - 1),  # lower left
        (x - 1, y + 1),  # upper left
        (x + 1, y),  # right
        (x + 1, y - 1),  # lower right
        (x + 1, y + 1),  # upper right
    ]


def update_level(x, y, levels, flashed):
    # This increases the energy level of all adjacent octopuses by 1,
    # including octopuses that are diagonally adjacent.
    for x2, y2 in get_adjacent(x, y):
        if in_range(x2, y2, levels):
            levels[y2][x2] += 1

            # If this causes an octopus to have an energy level greater than 9,
            # it also flashes. (An octopus can only flash at most once per step.)
            if levels[y2][x2] > 9 and (x2, y2) not in flashed:
                flashed.add((x2, y2))

                # This process continues as long as new octopuses keep having
                # their energy level increased beyond 9.
                flashed = update_level(x2, y2, levels, flashed)
    return flashed


def step(energy_levels: List[List[int]]) -> (int, List[List[int]]):
    """Returns number of flashes and the updated energy levels."""
    # First, the energy level of each octopus increases by 1.
    updated_levels = [[x + 1 for x in y] for y in energy_levels]

    # Then, any octopus with an energy level greater than 9 flashes.
    flashed = set()
    for j, y in enumerate(updated_levels):
        for i, x in enumerate(y):
            if x > 9:
                flashed.add((i, j))

    for x, y in flashed.copy():
        flashed.union(update_level(x, y, updated_levels, flashed))

    # Finally, any octopus that flashed during this step has its energy level set to 0,
    # as it used all of its energy to flash.
    for x, y in flashed:
        updated_levels[y][x] = 0
    return (len(flashed), updated_levels)


def solve_day11_part1(s: str, steps=100):
    energy_levels = parse_day11(s)
    total = 0
    for i in range(steps):
        flashes, energy_levels = step(energy_levels)
        total += flashes
    return int(total)


assert (res := solve_day11_part1(d11_example_txt, 2)) == 35, res
assert (res := solve_day11_part1(d11_example_txt, 10)) == 204, res
assert solve_day11_part1(d11_example_txt) == 1656

d11_input = """6318185732
1122687135
5173237676
8754362612
5718474666
8443654137
1247634346
1446514585
6717288267
1727871228"""

assert solve_day11_part1(d11_input) == 1634


def solve_day11_part2(s: str, steps=200):
    energy_levels = parse_day11(s)
    i = 0
    while True:
        flashes, energy_levels = step(energy_levels)
        i += 1
        if flashes == 100:
            break
    return i


assert (res := solve_day11_part2(d11_example_txt)) == 195, res
assert solve_day11_part2(d11_input) == 210


In [ ]:
# Day 12
from typing import List, Set
import itertools as it
from pprint import pprint
from collections import Counter

d12_example_txt = """start-A
start-b
A-c
A-b
b-d
A-end
b-end"""

d12_example_2_txt = """dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc"""

d12_example_3_txt = """fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW"""

d12_input = """um-end
pk-um
FE-il
ay-FE
pk-start
end-jt
um-FE
RO-il
xc-ay
il-end
start-EZ
pk-FE
xc-start
jt-FE
EZ-um
pk-xc
xc-EZ
pk-ay
il-ay
jt-EZ
jt-om
pk-EZ"""


def parse_day12(s: str) -> List[Set[str]]:
    return [set(x.split("-")) for x in s.splitlines()]


assert "start" in parse_day12(d12_example_txt)[0]


def find_connections(point: str, paths: List[Set[str]]) -> List[str]:
    connections = []
    for path in paths:
        if point in path:
            new_connection = (path - {point}).pop()
            connections.append(new_connection)
    return connections


assert find_connections("start", [{"start", "A"}, {"start", "b"}]) == ["A", "b"]


def traverse(path: List[str], paths: List[Set[str]], possible_paths=[]) -> int:
    if path[-1] == "end":
        possible_paths.append(path)
    else:
        for p in find_connections(path[-1], paths):
            new_path = path + [p]
            if not (p in path and p.islower()) and new_path not in possible_paths:
                possible_paths = traverse(new_path, paths, possible_paths)
    return possible_paths


def solve_day12_part1(s: str) -> int:
    possible_paths = traverse(["start"], parse_day12(s), [])
    # pprint(sorted(possible_paths))
    return len(possible_paths)


assert solve_day12_part1(d12_example_txt) == 10
assert (res := solve_day12_part1(d12_example_2_txt)) == 19, res
assert (res := solve_day12_part1(d12_example_3_txt)) == 226, res
assert solve_day12_part1(d12_input) == 3497


def traverse2(
    path: List[str], paths: List[Set[str]], possible_paths=[]
) -> int:
    if path[-1] == "end":
        possible_paths.append(path)
        # print(path)
    else:
        for p in find_connections(path[-1], paths):
            new_path = path + [p]
            if new_path in possible_paths:
                continue
            if p.islower():
                lower_letter_counts = Counter(filter(lambda x: x.islower(), path))
                if any(x>1 for x in lower_letter_counts.values()) and p in path:
                    continue
                if p == 'start':
                    continue
            possible_paths = traverse2(new_path, paths, possible_paths)
    return possible_paths


def solve_day12_part2(s: str) -> int:
    possible_paths = traverse2(["start"], parse_day12(s), [])
    return len(possible_paths)


assert (res := solve_day12_part2(d12_example_txt)) == 36, res
assert (res := solve_day12_part2(d12_example_2_txt)) == 103, res
assert (res := solve_day12_part2(d12_example_3_txt)) == 3509, res
# assert solve_day12_part2(d12_input) == 93686 # Warning: takes 17 minutes to run!


In [ ]:
# Day 13

from typing import Tuple, List
import pathlib

d13_example_txt = """6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5"""


def parse_day13(s: str) -> Tuple[List[Tuple[int, int]], List[Tuple[int, int]]]:
    coordinates, folds = s.split("\n\n")
    coordinates = [[int(i) for i in x.split(",")] for x in coordinates.splitlines()]
    folds = [
        [
            int(i) if i.isdigit() else 0 if i == "x" else 1
            for i in x.replace("fold along ", "").split("=")
        ]
        for x in folds.splitlines()
    ]
    return coordinates, folds


# parse_day13(d13_example_txt)


def fold(
    coordinates: List[Tuple[int, int]], folds: List[Tuple[int, int]], break_first=False
):
    for direction, position in folds:
        for coordinate in coordinates:
            if coordinate[direction] >= position:
                new_coord = position + (position - coordinate[direction])
                coordinate[direction] = new_coord
        if break_first:
            break
    return set(tuple(x) for x in coordinates)


assert (res := len(fold(*parse_day13(d13_example_txt), break_first=True))) == 17, res
p = pathlib.Path("data/AoC_2021_day_13.txt")
assert len(fold(*parse_day13(p.read_text()), break_first=True)) == 731

coordinates = sorted(
    list(fold(*parse_day13(p.read_text()))), key=lambda x: (x[1], x[0])
)
x_max = max(coordinates, key=lambda x: x[0])[0]
y_max = max(coordinates, key=lambda x: x[1])[1]

for j in range(y_max + 1):
    line = ""
    for i in range(x_max + 1):
        s = "."
        if (i, j) in coordinates:
            s = "#"
        line += s
    print(line)


In [ ]:
# Day 14
# %%prun -s tottime -l 15

from typing import Tuple, List, Dict
import pathlib
from itertools import starmap, chain, tee
from functools import reduce, lru_cache
from collections import Counter, defaultdict

d14_example_txt = """NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C"""


def parse_day14(s: str) -> Tuple[str, Dict[str, str]]:
    template, rules = s.split("\n\n")
    rules = [tuple(x.split(" -> ")) for x in rules.splitlines()]
    final_rules = [(tuple(x), y) for x, y in rules]
    return template, tuple(final_rules)


def pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)


def merge_dicts(dict1, dict2):
    dict3 = defaultdict(int)
    dict3.update(dict2)
    for k, v in dict1.items():
        dict3[k] += v
    return dict3


@lru_cache(maxsize=None)
def extend_pair(p, rules, remaining_steps):
    counts = defaultdict(int)
    ab = dict(rules)[p]
    counts[ab] = 1
    if remaining_steps == 1:
        return counts
    remaining_steps -= 1
    counts_ab = starmap(
        extend_pair,
        [((p[0], ab), rules, remaining_steps), ((ab, p[1]), rules, remaining_steps)],
    )
    return reduce(merge_dicts, chain(counts_ab, [counts]))


def solve_day14_part1(s: str) -> int:
    template, rules = parse_day14(s)
    counts = starmap(extend_pair, [(p, rules, 10) for p in pairwise(template)])
    counts = list(counts)

    elements = reduce(merge_dicts, chain(counts, [Counter(template)]))
    res = sorted(elements.values())
    return res[-1] - res[0]


assert (res := solve_day14_part1(d14_example_txt)) == 1588, res

p = pathlib.Path("data/AoC_2021_day_14.txt")
assert solve_day14_part1(p.read_text()) == 2321


def solve_day14_part2(s: str) -> int:
    template, rules = parse_day14(s)
    counts = starmap(extend_pair, [(p, rules, 40) for p in pairwise(template)])
    counts = list(counts)

    elements = reduce(merge_dicts, chain(counts, [Counter(template)]))
    res = sorted(elements.values())
    return res[-1] - res[0]


assert (res := solve_day14_part2(d14_example_txt)) == 2188189693529, res
assert solve_day14_part2(p.read_text()) == 2399822193707


In [ ]:
# Day 15

import networkx as nx
import pathlib
from copy import deepcopy

d15_example_txt = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""


def add_adjacent(G, p, floor_map):
    x, y = p
    right_x = x + 1
    if right_x < len(floor_map[0]):
        G.add_edge((x, y), (right_x, y), weight=floor_map[y][right_x])
    left_x = x - 1
    if left_x >= 0:
        G.add_edge((x, y), (left_x, y), weight=floor_map[y][left_x])
    below_y = y + 1
    if below_y < len(floor_map):
        G.add_edge((x, y), (x, below_y), weight=floor_map[below_y][x])
    above_y = y - 1
    if above_y >= 0:
        G.add_edge((x, y), (x, above_y), weight=floor_map[above_y][y])
    return G


def parse_day15(s: str) -> List[List[int]]:
    org_floor_map = []
    for y in s.splitlines():
        line = []
        for x in y:
            line.append(int(x))
        org_floor_map.append(line)
    return org_floor_map


def create_graph(floor_map):
    G = nx.DiGraph()
    for y in range(len(floor_map)):
        for x in range(len(floor_map[0])):
            add_adjacent(G, (x, y), floor_map)
    return G, (len(floor_map[0]) - 1, len(floor_map) - 1)


def solve_day15_part1(s: str):
    floor_map = parse_day15(s)
    G, target = create_graph(floor_map)
    source = (0, 0)
    return nx.astar_path_length(G, source, target)


assert solve_day15_part1(d15_example_txt) == 40

p = pathlib.Path("data/AoC_2021_day_15.txt")
assert solve_day15_part1(p.read_text()) == 386


def wraparound(x, r):
    res = x+r
    if (res) >= 10:
        return (res%10)+1
    return res

def repeat_floor_map(floor_map, repeat=5):
    org = deepcopy(floor_map)
    for y, line in enumerate(org):
        for r in range(1, repeat):
            floor_map[y].extend([wraparound(x, r) for x in line])
    lengthened_floor_map = deepcopy(floor_map)
    for r in range(1, repeat):
        floor_map.extend([[wraparound(x, r) for x in line] for line in lengthened_floor_map])
    
    return floor_map

def print_path(path):
    p, weight = path[0]
    x, y = p
    print(weight["weight"], end="")
    for p, weight in path[1:]:
        if p[1] > y:
            print("\n"+(" "*x), end="")
        print(weight["weight"], end="")
        x, y = p

def print_graph(G):
    y = 0
    x = 0
    print("1", end="")
    for node in sorted(G.nodes, key=lambda x: (x[1], x[0]))[1:]:
        if node[1] > y:
            print("")
            prev = (0, y)
        else:
            prev = (x, y)
        x, y = node
        # print(prev, node)
        print(G.edges[node, prev]["weight"], end="")
        # print(f"({x:02d},{y:02d})", end="")

def solve_day15_part2(s: str):
    floor_map = repeat_floor_map(parse_day15(s))
    # print(floor_map)
    G, target = create_graph(floor_map)
    print_graph(G)
    source = (0, 0)
    # print_path([(b, G.edges[a,b]) for a,b in pairwise(nx.astar_path(G, source, target))])
    return nx.astar_path_length(G, source, target)

assert (res := solve_day15_part2(d15_example_txt)) == 315, res

# solve_day15_part2(p.read_text())


In [ ]:
# Day 16
import itertools as it
import pathlib
from functools import reduce

def parse_day16(s: str):
    return list(map(int, it.chain(*[list(f"{int(h, 16):04b}") for h in s])))

assert (res := parse_day16("D2FE28")) == list(map(int, "110100101111111000101000")), res

def bin_list_to_int(bin_list):
    return sum([x*2**i for i, x in enumerate(reversed(bin_list))])

class Packet:
    def __init__(self, bin_list):
        self.version = bin_list_to_int(bin_list[:3])
        self.type = bin_list_to_int(bin_list[3:6])
        if self.type == 4:
            self.raw_value = []
            self.remaining = self.parse_literal(bin_list[6:])
            self.value = bin_list_to_int(self.raw_value)
        else:
            self.parse_operator(bin_list[6:])
            if self.type == 0:
                self.value = sum(map(lambda x: x.value, self.sub_packets))
            elif self.type == 1:
                self.value = reduce(lambda x, y: x*y, map(lambda x: x.value, self.sub_packets))
            elif self.type == 2:
                self.value = min(self.sub_packets, key=lambda x: x.value).value
            elif self.type == 3:
                self.value = max(self.sub_packets, key=lambda x: x.value).value
            elif self.type == 5:
                self.value = int(self.sub_packets[0].value > self.sub_packets[1].value)
            elif self.type == 6:
                self.value = int(self.sub_packets[0].value < self.sub_packets[1].value)
            elif self.type == 7:
                self.value = int(self.sub_packets[0].value == self.sub_packets[1].value)
        # print(self)
    
    def __str__(self):
        s = f"Packet(version={self.version}"
        if self.type == 4:
            s += f", type=literal, value={self.value}"
            if self.remaining:
                s+= f", remaining={''.join(map(str, self.remaining))}"
        else:
            operators = {
                0: "sum",
                1: "product",
                2: "minimum",
                3: "maximum",
                5: "greater than",
                6: "less than",
                7: "equal to"
            }
            s += f", type={operators[self.type]}, value={self.value}"
        
        return s + ")"
    
    def parse_operator(self, bin_list):
        self.length_type_id = bin_list[0]
        if self.length_type_id:
            self.num_sub_packets = bin_list_to_int(bin_list[1:1+11])
            self.sub_packets = self.parse_sub_packets(bin_list[1+11:], self.num_sub_packets)
            self.remaining = self.sub_packets[-1].remaining
        else:
            self.total_length = bin_list_to_int(bin_list[1:1+15])
            self.sub_packets = self.parse_sub_packets(bin_list[1+15:1+15+self.total_length])
            self.remaining = bin_list[1+15+self.total_length:]

    def parse_sub_packets(self, bin_list, num=0):
        p = Packet(bin_list)
        packets = [p]
        if not num:
            # Some bits left and they are not all zero
            while sum(p.remaining):
                p = Packet(p.remaining)
                packets.append(p)
        else:
            while len(packets) < num:
                p = Packet(p.remaining)
                packets.append(p)
        return packets

    def parse_literal(self, bin_list):
        self.raw_value += bin_list[1:5]
        if bin_list[0]:
            return self.parse_literal(bin_list[5:])
        else:
            return bin_list[5:]

    def sum_versions(self):
        v = self.version
        if hasattr(self, "sub_packets"):
            for p in self.sub_packets:
                v += p.sum_versions()
        return v

assert (res := Packet(parse_day16("D2FE28")).version) == 6, res
assert (res := Packet(parse_day16("D2FE28")).type) == 4, res
assert (res := Packet(parse_day16("D2FE28")).value) == 2021, res

assert (res := Packet(parse_day16("38006F45291200")).version) == 1, res
assert (res := Packet(parse_day16("38006F45291200")).type) == 6, res
assert (res := Packet(parse_day16("38006F45291200")).length_type_id) == 0, res
assert (res := Packet(parse_day16("38006F45291200")).total_length) == 27, res
assert (res := len(Packet(parse_day16("38006F45291200")).sub_packets)) == 2, res
assert (res := Packet(parse_day16("38006F45291200")).sub_packets[0].value) == 10, res
assert (res := Packet(parse_day16("38006F45291200")).sub_packets[1].value) == 20, res

assert (res := Packet(parse_day16("EE00D40C823060")).version) == 7, res
assert (res := Packet(parse_day16("EE00D40C823060")).type) == 3, res
assert (res := Packet(parse_day16("EE00D40C823060")).length_type_id) == 1, res
assert (res := Packet(parse_day16("EE00D40C823060")).num_sub_packets) == 3, res
assert (res := len(Packet(parse_day16("EE00D40C823060")).sub_packets)) == 3, res
assert (res := Packet(parse_day16("EE00D40C823060")).sub_packets[0].value) == 1, res
assert (res := Packet(parse_day16("EE00D40C823060")).sub_packets[1].value) == 2, res
assert (res := Packet(parse_day16("EE00D40C823060")).sub_packets[2].value) == 3, res

assert (res := Packet(parse_day16("8A004A801A8002F478")).sum_versions()) == 16, res
assert (res := Packet(parse_day16("620080001611562C8802118E34")).sum_versions()) == 12, res
assert (res := Packet(parse_day16("C0015000016115A2E0802F182340")).sum_versions()) == 23, res
assert (res := Packet(parse_day16("A0016C880162017C3686B18A3D4780")).sum_versions()) == 31, res

p = pathlib.Path("data/AoC_2021_day_16.txt")
assert Packet(parse_day16(p.read_text())).sum_versions() == 953

assert (res := Packet(parse_day16("C200B40A82")).value) == 3, res # 1+2
assert (res := Packet(parse_day16("04005AC33890")).value) == 54, res # 6*9
assert (res := Packet(parse_day16("880086C3E88112")).value) == 7, res # min(7,8,9)
assert (res := Packet(parse_day16("CE00C43D881120")).value) == 9, res # max(7,8,9)
assert (res := Packet(parse_day16("D8005AC2A8F0")).value) == 1, res # 5 < 15
assert (res := Packet(parse_day16("F600BC2D8F")).value) == 0, res # 5 > 15
assert (res := Packet(parse_day16("9C005AC2F8F0")).value) == 0, res # 5 == 15
assert (res := Packet(parse_day16("9C0141080250320F1802104A08")).value) == 1, res # 1 + 3 == 2 * 2

assert Packet(parse_day16(p.read_text())).value == 246225449979

In [ ]:
# Day 17
from collections import namedtuple
from typing import Tuple, List, Optional
from itertools import chain
import re

Coordinate = namedtuple("Coordinate", ["x", "y"])
Target = namedtuple("Target", ["left", "right", "low", "high"])

def parse_day17(s: str) -> Tuple[Coordinate, Coordinate]:
    return Target(*map(int, re.match(r"target area: x=([-\d]+)..([-\d]+), y=([-\d]+)..([-\d]+)", s).groups()))

assert parse_day17("target area: x=20..30, y=-10..-5") == Target(left=20, right=30, low=-10, high=-5)

def step(p: Coordinate, v: Coordinate) -> Tuple[Coordinate, Coordinate]:
    new_p = Coordinate(p.x+v.x, p.y+v.y)
    v_x = v.x
    if v.x > 0:
        v_x = v.x-1
    elif v.x < 0:
       v_x = v.x+1
    new_v = Coordinate(v_x, v.y-1)
    return new_p, new_v

def is_point_in_target(p: Coordinate, t: Target) -> bool:
    return (t.left <= p.x <= t.right) and (t.low <= p.y <= t.high)

def get_trajectory(v: Coordinate, t: Target) -> List[Coordinate]:
    trajectory = []
    p = Coordinate(0, 0)
    while not is_point_in_target(p, t) and p.y >= t.low:
        p, v = step(p, v)
        trajectory.append(p)
    if p.y < t.low:
        return []
    return trajectory

def max_y_of_trajectory(v, t) -> Optional[int]:
    trajectory = get_trajectory(v, t)
    if trajectory:
        return max(trajectory, key=lambda p: p.y).y
    return None

assert max_y_of_trajectory(Coordinate(6, 9), Target(left=20, right=30, low=-10, high=-5)) == 45

def solve_day17_part1(s: str):
    t = parse_day17(s)
    maxes = []
    for y in range(200):
        for x in range(200):
            max_y = max_y_of_trajectory(Coordinate(x, y), t)
            if max_y is not None:
                maxes.append(max_y)
    return max(maxes)

# assert solve_day17_part1("target area: x=20..30, y=-10..-5") == 45

# assert solve_day17_part1("target area: x=185..221, y=-122..-74") == 7381

def solve_day17_part2(s: str):
    t = parse_day17(s)
    inital_vs = []
    for y in range(-400, 400):
        for x in range(400):
            if get_trajectory(Coordinate(x, y), t):
                inital_vs.append((x,y))
    return len(inital_vs)

# assert (res := solve_day17_part2("target area: x=20..30, y=-10..-5")) == 112, res
assert solve_day17_part2("target area: x=185..221, y=-122..-74") == 3019